# Uncertainty Quantification Grid Operation

In Uncertainty Quantification (UQ), a problem function has parameters whose input is not exactly known. Each uncertain input parameter has a probability distribution. When evaluating the function, we can use the expectation as an approximate solution and the variance gives information about how much impact the uncertainty has on the solution. <br/>
With the `UncertaintyQuantification` grid operation, it is possible to can calculate moments, expectation and variance, and a polynomial chaos expansion (PCE). Input values which are not uncertain need to be fixed in the `Function` object. <br/>
The calculations involve weighted integration, where the integrated functions depend on the values which we want to calculate. For example, we can calculate the expectation and variance with the first and second moment:
$$
\mathrm{E} = \int_{w \in \Omega} f(w) p(w) dw \\
\mathrm{M}_2 = \int_{w \in \Omega} f(w)^2 p(w) dw \\
\mathrm{Var} = \mathrm{M}_2 - \mathrm{E}^2
$$
To reduce the number of problem function evaluations, the adaptive refinement generates a grid which fits well for multiple integrals, for example the first and second moments.
To this end, methods of `UncertaintyQuantification` concatenate relevant functions into a `Function` with multidimensional output. <br/>
`UncertaintyQuantification` supports the single dimension strategy with weighted global grids. In the following examples, the weighted trapezoidal grids are employed.


### Expectation and Variance Calculation

Here is an example for an expectation and variance calculation:

In [ ]:
%matplotlib inline
import numpy as np

import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *

# Let's select the three-dimensional discontinuous FunctionUQ
# as problem function and let the input parameters be
# normally distributed
problem_function = FunctionUQ()
dim = 3
distributions = [("Normal", 0.2, 1.0) for _ in range(dim)]

# a and b are the weighted integration domain boundaries.
# They should be set according to the distribution.
a = np.array([-np.inf] * dim)
b = np.array([np.inf] * dim)


# Create the grid operation and the weighted grid
op = UncertaintyQuantification(problem_function, distributions, a, b)
grid = GlobalTrapezoidalGridWeighted(a, b, op, boundary=False)

# The grid initialization requires the weight functions from the
# operation; since currently the adaptive refinement takes the grid from
# the operation, it has to be passed here
op.set_grid(grid)

# Select the function for which the grid is refined;
# here it is the expectation and variance calculation via the moments
op.set_expectation_variance_Function()

# Initialize the adaptive refinement instance and refine the grid until
# it has at least 200 points
combiinstance = SpatiallyAdaptiveSingleDimensions2(a, b, operation=op, norm=2, grid_surplusses=grid)
lmax = 2
error_operator = ErrorCalculatorSingleDimVolumeGuided()
combiinstance.performSpatiallyAdaptiv(1, lmax,
    error_operator, tol=0, max_evaluations=200, do_plot=True)


# Calculate the expectation and variance with the adaptive sparse grid
# weighted integral result
(E,), (Var,) = op.calculate_expectation_and_variance(combiinstance)
print(f"E: {E}, Var: {Var}")

### PCE Coefficient Calculation

In the pseudo-spectral PCE, the calculation of a coefficient $a_k$ requires the integration of a polynomial $q_k$ with norm $n_k$ multiplied with the problem function $f$:
$$
a_k = \frac{1}{n_k} \int_{w \in \Omega} f(w) q_k(w) p(w) dw
$$
Now we need a `Function` for the refinement so that all coefficients can be calculated with the resulting sparse grid.
`UncertaintyQuantification` internally uses Chaospy in various ways to calculate the PCE function.

In [ ]:
%matplotlib inline
import numpy as np

import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *

problem_function = FunctionUQ()
dim = 3
distributions = [("Normal", 0.2, 1.0) for _ in range(dim)]
a = np.array([-np.inf] * dim)
b = np.array([np.inf] * dim)
op = UncertaintyQuantification(problem_function, distributions, a, b)
grid = GlobalTrapezoidalGridWeighted(a, b, op, boundary=False)
op.set_grid(grid)


polynomial_degree_max = 2

# The grid needs to be refined for the PCE coefficient calculation
op.set_PCE_Function(polynomial_degree_max)


combiinstance = SpatiallyAdaptiveSingleDimensions2(a, b, operation=op, norm=2, grid_surplusses=grid)
lmax = 2
error_operator = ErrorCalculatorSingleDimVolumeGuided()
combiinstance.performSpatiallyAdaptiv(1, lmax,
    error_operator, tol=0, max_evaluations=200, do_plot=True)


# Create the PCE approximation; it is saved internally in the operation
op.calculate_PCE(None, combiinstance)

# Calculate the expectation, variance and sobol indices with the PCE coefficients
(E,), (Var,) = op.get_expectation_and_variance_PCE()
print(f"E: {E}, PCE Var: {Var}")
print("First order Sobol indices:", op.get_first_order_sobol_indices())
print("Total order Sobol indices:", op.get_total_order_sobol_indices())

### Custom Refinement Function

It is possible to choose an arbitrary function for the refinement. For example, we may be interested an accurate expectation and additionally print an approximate variance just for, let's say, debugging purposes. <br/>
To this end, we can take the code from the first example and change the refinement function and the `calculate_expectation_and_variance` invocation:

In [ ]:
%matplotlib inline
import numpy as np

import sparseSpACE
from sparseSpACE.Function import *
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *

problem_function = FunctionUQ()
dim = 3
distributions = [("Normal", 0.2, 1.0) for _ in range(dim)]
a = np.array([-np.inf] * dim)
b = np.array([np.inf] * dim)
op = UncertaintyQuantification(problem_function, distributions, a, b)
grid = GlobalTrapezoidalGridWeighted(a, b, op, boundary=False)
op.set_grid(grid)

# The refinement function for the expectation/first moment 
# is simply the problem function

combiinstance = SpatiallyAdaptiveSingleDimensions2(a, b, operation=op, norm=2, grid_surplusses=grid)
lmax = 2
error_operator = ErrorCalculatorSingleDimVolumeGuided()
combiinstance.performSpatiallyAdaptiv(1, lmax,
    error_operator, tol=0, max_evaluations=200, do_plot=False)

# Calculate the expectation and variance, and inform the method that
# the integral obtained during refinement cannot be used because it
# only has the expectation
(E,), (Var,) = op.calculate_expectation_and_variance(combiinstance,
    use_combiinstance_solution=False)

print(f"E: {E}, Var: {Var}")